# 06 correct lanes for reversible lanes with wrangler files

In [1]:
import os
import sys
import yaml
import pickle
import glob

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [47]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_05')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [48]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-02-26 13:07:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-26 13:07:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-26 13:07:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-26 13:07:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


## Read Version 05 Scenario

In [6]:
version_05_pickle_file_name = os.path.join(input_dir, 'working_scenario_05.pickle')
v_05_scenario = pickle.load(open(version_05_pickle_file_name, 'rb'))

In [7]:
v_05_scenario.road_net.links_df.shape

(2190435, 44)

In [8]:
v_05_scenario.road_net.links_df[
    v_05_scenario.road_net.links_df.model_link_id == 8016300].iloc[0].lanes

3

## Create Scenario and Apply Project Cards

In [9]:
v_06_scenario = Scenario.create_scenario(
    base_scenario= v_05_scenario,
    card_directory = card_dir,
    glob_search = "*.wrangler",
    validate_project_cards=False)

2021-02-24 18:21:00, INFO: Creating Scenario
2021-02-24 18:21:00, INFO: Creating Scenario
2021-02-24 18:21:01, INFO: Adding project cards using glob search: *.wrangler
2021-02-24 18:21:01, INFO: Adding project cards using glob search: *.wrangler


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [10]:
v_06_scenario.apply_all_projects()

2021-02-24 18:21:04, INFO: Applying year 2015 sterling on ramp HOV only in PM
2021-02-24 18:21:04, INFO: Applying year 2015 sterling on ramp HOV only in PM
2021-02-24 18:21:04, INFO: Applying Project to Roadway Network: year 2015 sterling on ramp HOV only in PM
2021-02-24 18:21:04, INFO: Applying Project to Roadway Network: year 2015 sterling on ramp HOV only in PM
2021-02-24 18:21:04, INFO: Applying year 2015 golden gate reversible lanes
2021-02-24 18:21:04, INFO: Applying year 2015 golden gate reversible lanes
2021-02-24 18:21:04, INFO: Applying Project to Roadway Network: year 2015 golden gate reversible lanes
2021-02-24 18:21:04, INFO: Applying Project to Roadway Network: year 2015 golden gate reversible lanes


In [11]:
v_06_scenario.road_net.links_df[
    v_06_scenario.road_net.links_df.model_link_id == 8016300].iloc[0].lanes

{'default': 3,
 'timeofday': [{'time': (21600, 36000), 'value': 4},
  {'time': (54000, 68400), 'value': 2}]}

In [12]:
v_06_scenario.road_net.links_df[
    v_06_scenario.road_net.links_df.model_link_id == 39115].iloc[0].useclass

{'default': 0, 'timeofday': [{'time': (54000, 68400), 'value': 3}]}

In [14]:
v_06_scenario.applied_projects

['Year 2015 Modify Facility Type to Service',
 'Year 2015 Modify Facility Type to Ramp shape incorrect',
 'Year 2015 Modify Facility Type to Ramp',
 'Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp shape incorrect',
 'Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp',
 'Year 2015 Modify Lanes to 1 Lane and Facility Type to Ramp',
 'Year 2015 Bus Only Segments',
 'Year 2015 Modify Lanes to 7 Lanes',
 'Year 2015 Modify Lanes to 6 Lanes',
 'Year 2015 Modify Lanes to 5 Lanes',
 'Year 2015 Modify Lanes to 4 Lanes',
 'Year 2015 Modify Lanes to 3 Lanes',
 'Year 2015 Modify Lanes to 2 Lanes',
 'Year 2015 Modify Lanes to 1 Lane',
 'Year 2015 Modify Lanes to 14 Lanes',
 'Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify HOV Lanes and USE on US 101 SB CA-85 to Oregon Expressway',
 'Year 2015 Modify HOV 

In [15]:
v_06_scenario.road_net.links_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2190435 entries, 0 to 2190434
Data columns (total 44 columns):
access                object
bike_access           object
drive_access          object
fromIntersectionId    object
lanes                 object
maxspeed              object
name                  object
oneWay                object
ref                   object
roadway               object
shstGeometryId        object
shstReferenceId       object
toIntersectionId      object
u                     object
v                     object
walk_access           object
wayId                 object
county                object
model_link_id         int64
A                     int64
B                     int64
rail_traveltime       object
rail_only             object
locationReferences    object
shape_id              object
geometry              geometry
ft_cal                object
ft                    int64
useclass              object
assignable            object
transit    

## Make Travel Model Networks

In [16]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_06_scenario.road_net, 
    parameters = parameters)

2021-02-24 18:27:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:27:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:27:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:27:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [17]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2190435, 44)
(688957, 12)
(1147816, 7)


In [18]:
model_net.links_df.model_link_id.nunique()

2190435

In [19]:
model_net = mtc.add_tap_and_tap_connector(
    roadway_network = model_net,
    parameters = parameters)

2021-02-24 18:27:51, INFO: Adding tap and tap connector to standard network
2021-02-24 18:27:51, INFO: Adding tap and tap connector to standard network
2021-02-24 18:27:51, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:27:51, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:27:51, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:27:51, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:27:56, INFO: Finished adding tap and tap connectors
2021-02-24 18:27:56, INFO: Finished adding tap and tap connectors


In [20]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2225677, 44)
(694957, 12)
(1165437, 7)


In [21]:
model_net.links_df.model_link_id.nunique()

2225677

In [22]:
model_net.nodes_df.model_node_id.nunique()

694957

In [23]:
model_net.shapes_df.id.nunique()

1165437

In [24]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2021-02-24 18:28:06, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-02-24 18:28:06, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-02-24 18:28:06, INFO: Creating managed lane network.
2021-02-24 18:28:06, INFO: Creating managed lane network.
2021-02-24 18:28:06, INFO: Creating network with duplicated managed lanes
2021-02-24 18:28:06, INFO: Creating network with duplicated managed lanes


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\wang

2021-02-24 18:44:08, INFO: Determining cntype
2021-02-24 18:44:08, INFO: Determining cntype
2021-02-24 18:44:08, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:44:08, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:44:08, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:44:08, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:44:09, INFO: Calculating and adding roadway network variable: cntype
2021-02-24 18:44:09, INFO: Calculating and adding roadway network variable: cntype
2021-02-24 18:45:30, INFO: Finished determining variable: cntype
2021-02-24 18:45:30, INFO: Finished determining variable: cntype
2021-02-24 18:45:30, INFO: Determining transit
2021-02-24 18:45:30, INFO: Determining transit
2021-02-24 18:45:30, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-24 18:45:30, INFO: Lasso base directory set a

In [25]:
model_net.links_df.shape

(2230076, 57)

In [26]:
model_net.links_df.model_link_id.nunique()

2230076

In [27]:
model_net.nodes_mtc_df.shape

(696462, 13)

In [28]:
model_net.nodes_df.model_node_id.nunique()

696462

## Write to Disk

## As shapefile

In [41]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry", 
                             "drive_access",
                            'lanes_EA', 'lanes_AM',
       'lanes_MD', 'lanes_PM', 'lanes_EV', "useclass_AM", "useclass_MD", 
                            "useclass_PM", "useclass_EV", "useclass_EA",
                            "cntype"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2021-02-24 20:35:07, INFO: Writing Network as Shapefile
2021-02-24 20:35:07, INFO: Writing Network as Shapefile
2021-02-24 20:35:07, INFO: Renaming DBF Node Variables
2021-02-24 20:35:07, INFO: Renaming DBF Node Variables
2021-02-24 20:35:07, INFO: Renaming variables so that they are DBF-safe
2021-02-24 20:35:07, INFO: Renaming variables so that they are DBF-safe
2021-02-24 20:35:20, INFO: Renaming DBF Link Variables
2021-02-24 20:35:20, INFO: Renaming DBF Link Variables
2021-02-24 20:35:20, INFO: Renaming variables so that they are DBF-safe
2021-02-24 20:35:20, INFO: Renaming variables so that they are DBF-safe
2021-02-24 20:35:24, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_04\nodes.shp
2021-02-24 20:35:24, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_04\nodes.shp
2021-02-24 20:36:10, INFO: Writing Link Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\pro

## As fixed width (for CUBE)

In [30]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2021-02-24 19:07:30, INFO: Starting fixed width conversion
2021-02-24 19:07:30, INFO: Starting fixed width conversion
2021-02-24 19:34:15, INFO: Writing out link database
2021-02-24 19:34:15, INFO: Writing out link database
2021-02-24 19:34:31, INFO: Writing out link header and width ----
2021-02-24 19:34:31, INFO: Writing out link header and width ----
2021-02-24 19:34:31, INFO: Starting fixed width conversion
2021-02-24 19:34:31, INFO: Starting fixed width conversion
2021-02-24 19:36:37, INFO: Writing out node database
2021-02-24 19:36:37, INFO: Writing out node database
2021-02-24 19:36:39, INFO: Writing out node header and width
2021-02-24 19:36:39, INFO: Writing out node header and width


In [31]:
%%capture
os.chdir(output_dir)
!make_cube.bat


(lasso_mtc) D:\github\lfs-cleaning\travel-model-two-networks\data\processed\version_04>SET LINK_DATA_PATH=links.txt 

(lasso_mtc) D:\github\lfs-cleaning\travel-model-two-networks\data\processed\version_04>SET NODE_DATA_PATH=nodes.txt 

(lasso_mtc) D:\github\lfs-cleaning\travel-model-two-networks\data\processed\version_04>SET SCENARIO_DIR=. 

(lasso_mtc) D:\github\lfs-cleaning\travel-model-two-networks\data\processed\version_04>SET zones=4756 

(lasso_mtc) D:\github\lfs-cleaning\travel-model-two-networks\data\processed\version_04>runtpp make_complete_network_from_fixed_width_file.s 

PILOT    (v.10/06/2017 [6.4.3 x86]) Wed Feb 24 19:36:45 2021

Print:   D:...el-model-two-networks\data\processed\version_04\TPPL0013.PRN
Input:   D:...sed\version_04\make_complete_network_from_fixed_width_file.s
NETWORK  (v.10/06/2017 [6.4.3 x86]) Wed Feb 24 19:36:47 2021

nodes.txt(1000):   65356861.0;San Francisco;1;1;1;1000048;0; 4758737.63344123
nodes.txt(2000):   53069265.0;      Alameda;1;1;1;2500400

In [32]:
%%capture
!runtpp make_simple_roadway_network.s
!runtpp make_complete_marin_network.s
!runtpp make_simple_transit_network.s
!runtpp make_highway_ext_stn_connector_network.s

nodes.txt(29000):  315024462.0;  Santa Clara;0;1;1;2009042;0; 4899782.5180840
nodes.txt(30000):   53059953.0;      Alameda;1;1;1;2505810;0; 4899484.1588253
nodes.txt(31000):   53019123.0;      Alameda;1;1;1;2506012;0; 4812721.2443035
nodes.txt(32000): 5271196140.0;  Santa Clara;0;1;0;2010039;0;  4904631.757923
nodes.txt(33000): 3742144548.0;    San Mateo;1;1;1;1502864;0;4792408.62454314
nodes.txt(34000):   55955673.0;         Napa;1;1;1;4000653;0; 4831443.2082325
nodes.txt(35000): 4600862747.0; Contra Costa;1;1;1;3005145;0; 4870750.6994182
nodes.txt(36000): 1511926467.0;  Santa Clara;1;1;1;2011230;0; 4980636.4920916
nodes.txt(37000): 1847656857.0;  Santa Clara;1;1;1;2011540;0; 4884169.8078125
nodes.txt(38000):   65590680.0;  Santa Clara;1;1;1;2011880;0; 4871497.6298627
nodes.txt(39000): 4658078695.0;         Napa;0;1;0;4000753;0; 4809698.7311803
nodes.txt(40000): 1673754749.0; Contra Costa;1;1;1;3005876;0; 4821461.4707896
nodes.txt(41000): 4930618917.0; Contra Costa;1;1;1;3006045;0; 48

links.txt(445000): 4510157;4527764;0;1;0;       Sonoma;  0.030837059517638554
links.txt(446000): 3525881;3504234;1;1;0;       Solano;   0.07760875192582256
links.txt(447000): 2571349;2512776;1;1;0;      Alameda;   0.04689943283039492
links.txt(448000): 2113882;2113305;1;1;0;  Santa Clara;   0.05253164399431796
links.txt(449000): 2114063;2134851;0;1;0;  Santa Clara;   0.04110689234232844
links.txt(450000): 2571727;2585015;0;0;0;      Alameda;  0.003309254520567668
links.txt(451000): 2528594;2617961;1;1;0;      Alameda; 0.0046837221920769025
links.txt(452000): 4519204;4550128;1;1;0;       Sonoma;   0.05750963592969951
links.txt(453000): 4007299;4007518;0;0;0;         Napa;   0.04799026512928003
links.txt(454000): 3526246;3543584;1;1;0;       Solano;  0.015724499565149045
links.txt(455000): 3012840;3043403;0;1;0; Contra Costa;  0.008673371976906217
links.txt(456000): 4007349;4006952;1;1;0;         Napa;   0.09746169300496174
links.txt(457000): 2115604;2046907;0;0;0;  Santa Clara;   0.0154

links.txt(690000): 2594564;2533318;0;1;0;      Alameda;  0.029176645186433085
links.txt(691000): 5018997;5019384;1;1;0;        Marin;   0.13553927756826842
links.txt(692000): 2152930;2123568;0;0;0;  Santa Clara;   0.09873041286931696
links.txt(693000): 3535057;3535564;0;1;0;       Solano;  0.002649462811739895
links.txt(694000): 3072127;3027129;1;1;0; Contra Costa;  0.007939808033066726
links.txt(695000): 1022297;1012403;1;1;0;San Francisco;   0.00681929925134565
links.txt(696000): 2126240;2188790;0;1;0;  Santa Clara;   0.04556685241364777
links.txt(697000): 2085278;2090481;0;1;0;  Santa Clara;   0.07942731358524402
links.txt(698000): 2153687;2148034;0;1;0;  Santa Clara;   0.07140511555477426
links.txt(699000): 2597310;2622693;1;1;0;      Alameda;  0.022316232036673395
links.txt(700000): 2153938;2093525;1;1;0;  Santa Clara;   0.08607035892719889
links.txt(701000): 4530714;4553648;0;1;0;       Sonoma;   0.01876567529663295
links.txt(702000): 3012574;3036747;1;1;0; Contra Costa;  0.01398

links.txt(1513000): 2173899;2029615;0;1;0;  Santa Clara;  0.02949570685845548
links.txt(1514000): 2021886;2153155;0;0;0;  Santa Clara;   0.0055555681342310
links.txt(1515000): 3021859;3077718;1;1;0; Contra Costa;   0.0512597717496035
links.txt(1516000): 2578506;2562578;0;1;0;      Alameda;  0.01094906887993101
links.txt(1517000): 3000036;3047938;1;1;0; Contra Costa;   0.0489613138005126
links.txt(1518000): 2065516;2142233;0;1;0;  Santa Clara;  0.03631509450307876
links.txt(1519000): 5023084;5010061;0;0;0;        Marin;   0.0178423692292540
links.txt(1520000): 5014736;5020643;1;1;0;        Marin;   0.0286980776610968
links.txt(1521000): 2603980;2563110;1;1;0;      Alameda;   0.0390062948650354
links.txt(1522000): 2587709;2508096;1;1;0;      Alameda;   0.0182637174001295
links.txt(1523000): 3007306;3011465;1;1;0; Contra Costa;   0.0732945460822705
links.txt(1524000): 1000604;1021404;1;1;0;San Francisco;  0.05567295529805485
links.txt(1525000): 2134313;2118567;0;0;0;  Santa Clara;   0.072

links.txt(2070000):  328526;2548337;0;1;0;      Alameda;   0.0734380033182071
links.txt(2071000):  323621;2550557;0;1;0;      Alameda;   0.0317710857076567
links.txt(2072000):  310242;2552857;0;1;0;      Alameda;   0.0090885219514905
links.txt(2073000):  327109;2555255;0;1;0;      Alameda;   0.0707993071375837
links.txt(2074000):  329722;2557580;0;1;0;      Alameda;  0.04236224819391222
links.txt(2075000):  331748;2559920;0;1;0;      Alameda;   0.0573443812676896
links.txt(2076000):  315094;2562248;0;1;0;      Alameda;  0.06106126336204823
links.txt(2077000):  312551;2564615;0;1;0;      Alameda;   0.0632002650845616
links.txt(2078000):  313555;2566930;0;0;0;      Alameda;   0.1275431704101197
links.txt(2079000):  310587;2569312;0;1;0;      Alameda;    0.035957265684717
links.txt(2080000):  322273;2571652;0;1;0;      Alameda;   0.0867234308583586
links.txt(2081000):  316848;2574027;0;0;0;      Alameda;   0.1069284271799933
links.txt(2082000):  315559;2576301;0;1;0;      Alameda;   0.056


NODEMERGE: 2604138

NODEMERGE: 2605138

NODEMERGE: 2606138

NODEMERGE: 2607138

NODEMERGE: 2608138

NODEMERGE: 2609138

NODEMERGE: 2610138

NODEMERGE: 2611138

NODEMERGE: 2612138

NODEMERGE: 2613138

NODEMERGE: 2614138

NODEMERGE: 2615138

NODEMERGE: 2616138

NODEMERGE: 2617138

NODEMERGE: 2618138

NODEMERGE: 2619138

NODEMERGE: 2620138

NODEMERGE: 2621138

NODEMERGE: 2622138

NODEMERGE: 2623138

NODEMERGE: 2624138

NODEMERGE: 2625138

NODEMERGE: 3000163

NODEMERGE: 3001163

NODEMERGE: 3002163

NODEMERGE: 3003163

NODEMERGE: 3004163

NODEMERGE: 3005163

NODEMERGE: 3006163

NODEMERGE: 3007163

NODEMERGE: 3008163

NODEMERGE: 3009163

NODEMERGE: 3010163

NODEMERGE: 3011163

NODEMERGE: 3012163

NODEMERGE: 3013163

NODEMERGE: 3014163

NODEMERGE: 3015163

NODEMERGE: 3016163

NODEMERGE: 3017163

NODEMERGE: 3018163

NODEMERGE: 3019163

NODEMERGE: 3020163

NODEMERGE: 3021163

NODEMERGE: 3022163

NODEMERGE: 3023163

NODEMERGE: 3024163

NODEMERGE: 3025163

NODEMERGE: 3026163

NODEMERGE: 3027163


## As transit line file (for CUBE)

In [50]:
mtc.write_cube_fare_files(
    roadway_network=v_06_scenario.road_net,
    transit_network=v_06_scenario.transit_net,
    parameters=parameters,
    outpath = os.path.join(output_dir)
)

In [43]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_06_scenario.transit_net, parameters = parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2021-02-26 12:46:19, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-26 12:46:19, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-26 12:46:19, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-26 12:46:19, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-26 12:46:19, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-02-26 12:46:19, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


## As pickle

In [35]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_06.pickle')
pickle.dump(v_06_scenario, open(working_scenario_filename, 'wb'))

## As standard network

In [36]:
model_net.write(path = output_dir, filename = "v05")